In [1]:
import polars as pl

In [2]:
df = pl.read_parquet("diagnosticos.parquet")


In [5]:
import polars as pl
from pathlib import Path

# --- CONFIGURAÇÃO ---
# Defina o caminho para a pasta onde os arquivos .parquet estão localizados.
# Assumindo que o notebook está no mesmo diretório dos arquivos.
# Se não estiver, ajuste o caminho. Ex: DATA_DIR = Path("caminho/para/pasta/processed")
DATA_DIR = Path(".") 

# Nomes dos arquivos
INTERNACOES_FILE = "internacoes.parquet"
MUNICIPIOS_FILE = "municipios.parquet"

# --- ANÁLISE ---
print("Iniciando a análise de inconsistência de códigos de município...")

try:
    # 1. Carregar os DataFrames de forma "lazy" (preguiçosa) para economizar memória
    print(f"Carregando '{INTERNACOES_FILE}'...")
    df_internacoes_lazy = pl.scan_parquet(DATA_DIR / INTERNACOES_FILE)

    print(f"Carregando '{MUNICIPIOS_FILE}'...")
    df_municipios_lazy = pl.scan_parquet(DATA_DIR / MUNICIPIOS_FILE)

    # 2. Selecionar apenas as colunas de interesse para a análise
    # Seleciona a coluna MUNIC_RES e garante que ela seja do tipo String
    internacoes_codigos = df_internacoes_lazy.select(
        pl.col("MUNIC_RES").cast(pl.String)
    ).unique()

    # Seleciona a coluna codigo_6d e garante que ela seja do tipo String
    municipios_codigos = df_municipios_lazy.select(
        pl.col("codigo_6d").cast(pl.String)
    ).unique()

    # 3. Executar o ANTI-JOIN
    # Esta é a operação principal: ela encontra todos os MUNIC_RES em 'internacoes_codigos'
    # que NÃO existem em 'municipios_codigos'.
    print("Executando anti-join para encontrar códigos órfãos...")
    codigos_orfaos = internacoes_codigos.join(
        municipios_codigos,
        left_on="MUNIC_RES",
        right_on="codigo_6d",
        how="anti"
    )

    # Coleta o resultado (traz os dados para a memória)
    resultado_orfaos = codigos_orfaos.collect()

    # 4. Apresentar os resultados
    if resultado_orfaos.is_empty():
        print("\n" + "="*50)
        print("✅ SUCESSO! Todos os códigos 'MUNIC_RES' foram encontrados na tabela de municípios.")
        print("="*50)
    else:
        print("\n" + "="*50)
        print(f"🚨 ALERTA! Encontrados {len(resultado_orfaos)} códigos 'MUNIC_RES' órfãos (sem correspondência).")
        print("="*50)
        print("Amostra dos códigos não encontrados:")
        print(resultado_orfaos.head(20)) # Mostra os primeiros 20 códigos órfãos

        # Opcional: Contar a frequência de cada código órfão para ver os mais problemáticos
        print("\nContando a frequência de cada código órfão no arquivo de internações completo...")
        
        # Re-lê o arquivo de internações para contar as ocorrências
        contagem_orfaos = (
            pl.scan_parquet(DATA_DIR / INTERNACOES_FILE)
            .filter(pl.col("MUNIC_RES").is_in(resultado_orfaos["MUNIC_RES"]))
            .group_by("MUNIC_RES")
            .agg(pl.count().alias("numero_de_ocorrencias"))
            .sort("numero_de_ocorrencias", descending=True)
            .collect()
        )
        
        print("\nCódigos órfãos mais frequentes:")
        print(contagem_orfaos.head(20))

except FileNotFoundError as e:
    print(f"\nERRO: Arquivo não encontrado. Verifique se os nomes e o caminho estão corretos.")
    print(f"Detalhe do erro: {e}")
except Exception as e:
    print(f"\nOcorreu um erro inesperado durante a análise: {e}")



Iniciando a análise de inconsistência de códigos de município...
Carregando 'internacoes.parquet'...
Carregando 'municipios.parquet'...
Executando anti-join para encontrar códigos órfãos...

🚨 ALERTA! Encontrados 11 códigos 'MUNIC_RES' órfãos (sem correspondência).
Amostra dos códigos não encontrados:
shape: (11, 1)
┌───────────┐
│ MUNIC_RES │
│ ---       │
│ str       │
╞═══════════╡
│ 530100    │
│ 530030    │
│ 530120    │
│ 530040    │
│ 530150    │
│ …         │
│ 530140    │
│ 530130    │
│ 530070    │
│ 530180    │
│ 530050    │
└───────────┘

Contando a frequência de cada código órfão no arquivo de internações completo...

Códigos órfãos mais frequentes:
shape: (11, 2)
┌───────────┬───────────────────────┐
│ MUNIC_RES ┆ numero_de_ocorrencias │
│ ---       ┆ ---                   │
│ str       ┆ u32                   │
╞═══════════╪═══════════════════════╡
│ 530180    ┆ 11                    │
│ 530040    ┆ 6                     │
│ 530070    ┆ 5                     │
│ 530030  

/tmp/ipykernel_40495/4099976251.py:70: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("numero_de_ocorrencias"))
/tmp/ipykernel_40495/4099976251.py:72: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  .collect()


In [2]:
import polars as pl
from pathlib import Path

# --- CONFIGURAÇÃO ---
# Caminho para o arquivo que sai do pré-processamento
PREPROCESSED_FILE = Path("internacoes.parquet")

print(f"Iniciando verificação de consistência interna do arquivo: '{PREPROCESSED_FILE}'")
print("="*80)

try:
    # Usamos scan_parquet para ser eficiente com a memória
    lazy_df = pl.scan_parquet(PREPROCESSED_FILE)
    total_registros = lazy_df.select(pl.len()).collect().item()
    print(f"Arquivo encontrado. Total de registros a serem verificados: {total_registros:,}")

    # --- Verificação de Consistência ---
    
    # Filtra o DataFrame para encontrar linhas onde DIAS_PERM não bate com o cálculo das datas
    # Usamos .dt.total_days() para extrair o número de dias da diferença
    df_inconsistentes = lazy_df.filter(
        pl.col("DIAS_PERM") != (pl.col("DT_SAIDA") - pl.col("DT_INTER")).dt.total_days()
    ).collect() # .collect() executa a consulta e materializa o resultado

    total_inconsistentes = len(df_inconsistentes)

    print("-" * 80)
    if total_inconsistentes == 0:
        print("\n✅ SUCESSO! O arquivo 'internacoes.parquet' está 100% consistente.")
        print("   A coluna 'DIAS_PERM' corresponde perfeitamente ao cálculo (DT_SAIDA - DT_INTER).")
        print("   Isso sugere que a inconsistência está sendo introduzida na etapa de AGREGAÇÃO ('aggregate.py').")
    else:
        print(f"\n🚨 ALERTA! Encontrados {total_inconsistentes:,} registros inconsistentes dentro do próprio arquivo 'sih_rs_tratado.parquet'.")
        print("   Isso indica um problema na lógica de recálculo dentro do 'preprocess.py'.")
        
        # Adiciona a coluna de diferença para facilitar a análise
        df_inconsistentes = df_inconsistentes.with_columns(
            (pl.col("DIAS_PERM") - (pl.col("DT_SAIDA") - pl.col("DT_INTER")).dt.total_days()).alias("diferenca")
        )
        
        print("\n--- Amostra dos dados inconsistentes encontrados ---")
        print(df_inconsistentes.select([
            "N_AIH", "DT_INTER", "DT_SAIDA", "DIAS_PERM", "diferenca"
        ]).head(20))

except FileNotFoundError:
    print(f"\nERRO: Arquivo '{PREPROCESSED_FILE}' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")



Iniciando verificação de consistência interna do arquivo: 'internacoes.parquet'
Arquivo encontrado. Total de registros a serem verificados: 11,821,898
--------------------------------------------------------------------------------

✅ SUCESSO! O arquivo 'internacoes.parquet' está 100% consistente.
   A coluna 'DIAS_PERM' corresponde perfeitamente ao cálculo (DT_SAIDA - DT_INTER).
   Isso sugere que a inconsistência está sendo introduzida na etapa de AGREGAÇÃO ('aggregate.py').


In [ ]:
import polars as pl

# Aponte para o arquivo que é a FONTE da sua tabela 'internacoes'
# Geralmente é o 'internacoes.parquet' na pasta 'processed'
ARQUIVO_FINAL = "internacoes.parquet" 

print(f"Verificando a consistência final do arquivo: '{ARQUIVO_FINAL}'")

df = pl.read_parquet(ARQUIVO_FINAL)

# Recalcula a idade usando a mesma lógica do aggregate.py
df_verificado = df.with_columns(
    (
        pl.col("DT_INTER").dt.year() - pl.col("NASC").dt.year() -
        pl.when(
            (pl.col("DT_INTER").dt.month() < pl.col("NASC").dt.month()) |
            ((pl.col("DT_INTER").dt.month() == pl.col("NASC").dt.month()) &
             (pl.col("DT_INTER").dt.day() < pl.col("NASC").dt.day()))
        )
        .then(1)
        .otherwise(0)
    )
    .cast(pl.Int16)
    .alias("idade_recalculada")
)

# Encontra as linhas onde a IDADE no arquivo não bate com o recálculo
inconsistencias = df_verificado.filter(
    pl.col("IDADE") != pl.col("idade_recalculada")
)

if inconsistencias.is_empty():
    print("\n✅ SUCESSO! O arquivo Parquet final está 100% consistente. O problema NÃO está no pipeline Python.")
else:
    print(f"\n🚨 FALHA! Encontradas {len(inconsistencias)} inconsistências no arquivo Parquet final. O problema está no 'aggregate.py'.")
    print("Amostra das falhas:")
    print(inconsistencias)
